# Normal modes of a 1D system with anisotropy and DMI

**Purpose**: Compute normal modes of a 1D system with anisotropy and DMI and compare the computed solutions to analytical values.

**Author and creation date**: Weiwei Wang, Dec 2013

**Modifications by**:

In this example we test the normal modes using a 1d mesh. We consider exchange, anisotropy and DMI interactions. The expected frequencies are
$$\frac{\omega}{\gamma} = K + D \left( \frac{n \pi}{L} \right)^2$$
where 
$$K = \frac{2 K_x}{\mu_0 M_s} \qquad  D= \frac{2 A}{\mu_0 M_s}$$

In [10]:
import numpy as np
np.set_printoptions(precision=10)  # limit precision for printing to make the notebook reproducible for the tests

In [12]:
def compute_modes_analytically(ns):
    L = 300e-9
    A = 1.3e-11
    Kx = 8e4
    gamma=2.21e5
    Ms = 8.0e5
    Dx = 4e-4
    mu0 = 4*np.pi*1e-7
    J = 2*A/(mu0*Ms)
    D = 2*Dx/(mu0*Ms)
    K = 2*Kx/(mu0*Ms)
    freqs = []
    for i in range(ns):
        w=gamma*(K+J*(i*np.pi/L)**2-0.25*D**2/J)/(2*np.pi)
        freqs.append(w/1e9)
    return np.array(freqs)

In [13]:
import dolfin as df
from finmag import Simulation
from finmag.energies import Exchange, DMI, Demag, Zeeman, UniaxialAnisotropy
from finmag.sim.normal_mode_sim import NormalModeSimulation
from finmag.util.helpers import set_logging_level
set_logging_level("INFO")

In [15]:
def compute_normal_modes(mesh, ns=10):
    Ms = 8.0e5
    sim = NormalModeSimulation(mesh, Ms, unit_length=1e-9, name = 'relax' )

    sim.set_m((1,0,0))
    A = 1.3e-11
    Kx = 8e4
    Dx = 4e-4

    sim.add(Exchange(A))
    sim.add(DMI(Dx))
    sim.add(UniaxialAnisotropy(Kx,axis=[1,0,0], name='Kx'))

    omega, w, rel_errors = sim.compute_normal_modes(n_values=ns, solver='scipy_sparse')
    assert np.allclose(rel_errors, 0, atol=1e-11)
    omega = np.real_if_close(omega, tol=1e3)
    freqs = [f for f in omega if f>0]
    
    return np.array(sorted(freqs))

In [16]:
mesh = df.IntervalMesh(100, -100, 200)
freqs = compute_normal_modes(mesh)
print freqs
freqs2 = compute_modes_analytically(5)
print freqs2

[2014-06-09 18:03:28] INFO: Finmag logging output will be written to file: '/home/albert/work/code/finmag/doc/ipython_notebooks_src/relax.log' (any old content will be overwritten).
[2014-06-09 18:03:28] INFO: Creating Sim object 'relax' (rank=0/1).
[2014-06-09 18:03:28] INFO: <Mesh of topological dimension 1 (intervals) with 101 vertices and 100 cells, ordered>
[2014-06-09 18:03:28] INFO: Assembling eigenproblem matrix.
[2014-06-09 18:03:29] INFO: Finished assembling eigenproblem matrix.
[2014-06-09 18:03:29] INFO: Solving eigenproblem. This may take a while...
[2014-06-09 18:03:29] INFO: Computing the eigenvalues and eigenvectors took 0.02 seconds


[  5.3832237919   5.4830865318   5.7825761991   6.2813972336   6.9790573592
   7.8748680696   8.9679453077  10.2572103383  11.7413908125  13.4190220233]
[ 5.382687881   5.4824448254  5.7817156588  6.280500381   6.9787989921]


As we can see, the frequencies obtained using two methods are quite close.